<a href="https://colab.research.google.com/github/ordavidov/ocl_lab/blob/aaai/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/ocl_lab/notebooks/DOFramework

# DOFramework Setup

*****

We begin by installing the Python package [`doframework`](https://github.com/IBM/doframework). If you are getting errors, restart the runtime and run this cell again (the errors are due to mismatches between Google Colab built-in environment packages and `doframework` requirements that cannot be automatically resolved).

In [ ]:
%pip install doframework

In [ ]:
from dataclasses import dataclass, field, InitVar
from typing import Any
import itertools as it
import os
import shutil

import numpy as np
from scipy.spatial import ConvexHull
from scipy.stats import uniform
from sklearn.linear_model import LinearRegression

from doframework.core.pwl import PWL
from doframework.core.sampler import D_sampler as sampler
from doframework.core.triangulation import box_iterator
from doframework.core.hit_and_run import in_domain
from doframework.core.utils import sample_standard_simplex, incidence
from doframework.core.storage import Storage
from doframework.core.inputs import get_configs

`doframework` relies on [`rayvens`](https://github.com/project-codeflare/rayvens) for event streaming over [`camel`](https://github.com/apache/camel-k/releases?page=3) and on the event distribution framework [`ray`](https://www.ray.io/). To use the `camel` framework, we need to install it together with [JDK](https://www.oracle.com/java/technologies/downloads/) and [`maven`](https://maven.apache.org/).

In [ ]:
!wget https://github.com/apache/camel-k/releases/download/v1.5.1/camel-k-client-1.5.1-linux-64bit.tar.gz
!tar zxvf camel-k-client-1.5.1-linux-64bit.tar.gz
!cp ./kamel /usr/local/bin
!kamel version

def install_java():

  !apt update
  !apt-get install -y openjdk-11-jdk-headless -qq
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
  !apt-get install -y maven 
  !java -version
  !mvn -version
  
install_java()

In [ ]:
tol = 1e-8 # tolerance to near 0

In [ ]:
def triangulate(fpoints: np.array, opoints: np.array, fvals: np.array, ovals: np.array):
    
    assert all([fpoints.shape[0]==fvals.flatten().size,opoints.shape[0]==ovals.flatten().size]), 'Length of values array should match the number of row vectors.'

    m = fpoints.min()
    M = fpoints.max()

    olift = np.hstack([opoints,(np.random.rand(opoints.shape[0])*(M-m)+m)[:,None]])
    flift = np.hstack([fpoints,(np.random.rand(fpoints.shape[0])*(M-m)+11*(M-m)+m)[:,None]]) 

    P = np.vstack([opoints,fpoints])
    _, unique_indices = np.unique(P, axis=0, return_index=True) 
    Plift = np.vstack([olift,flift])[unique_indices]

    view_point = np.concatenate([P.mean(axis=0),np.array([m-1000*(M-m)])]) 
    envelope = ConvexHull(np.vstack([np.atleast_2d(view_point),Plift]),qhull_options='QG0')
    good_indices = envelope.simplices[envelope.good]
    fPs = envelope.points[good_indices,:][:,:,:-1]

    V = np.concatenate([ovals,fvals])[unique_indices]
    fVs = V[:,None][good_indices-1].reshape(*good_indices.shape) # view point at index 0

    oin = [np.all(incidence(opoints,fp).any(axis=0)) for fp in fPs]
    oPs = fPs[oin]
    oVs = fVs[oin]

    if oPs.size == 0: # when fail to catch omega lower envelope
        oPs, oVs = fPs, fVs

    return fPs, fVs, oPs, oVs

# DOFramework Example

*****

This notebook will demo ```doframework``` on a naive Optimization with Constraint Learning (OCL) algorithm. Ideally, ```doframework``` will be used against a more sophisticated OCL algorithm, such as [OptiCL](https://github.com/hwiberg/OptiCL), to check its effectiveness. 

`doframework` randomly generates optimization problem instances $(f,\Omega,D,\mathbf{x}^*)$ for the OCL algorithm to solve. These optimization problem instances include:
* $f: \mathbb{R}^d → \mathbb{R}$ a continuous piece-wise linear objective.
* $\Omega ⊆ \mathbb{R}^d$ a feasibility region as a bounded convex $d$-polytope.
* $D = (X,y)$ data associated with $f$ so that $X ⊆ \mbox{dom}(f)$ and $y = f(\mathbf{x}) + ϵ$, $ϵ \sim \mathcal{N}(0,σ^2)$.
* $\mathbf{x}^* = \arg \min_{\mathbf{x} \in Ω} f(\mathbf{x})$ the ground-truth optimum.

`doframework` feeds $(\Omega,D)$ to a user-provided OCL algorithm. It then collects its predicted optimum $\hat{\mathbf{x}}^*$ to compare against $\mathbf{x}^*$.

This notebook is divided into two parts. In <font color=green>Part I</font> we will define a naive OCL algorithm and test it. Here, we will work with a PWL object, which is the fundamental object ```doframework``` uses to generate constraints and data.

Once we have tested our OCL algorithm, we will switch to <font color=green>Part II</font>, where we will demonstrate running ```doframework``` on our algorithm.

# Part I

*****

## -- Objective

We will first define a _test_ objective target to use against our naive OCL algorithm. We will define the objective target as a PWL object, similarly to the way ```doframework``` does it (only for more sophisticated PWL functions).

The domain of the objective function we'll use will be a ```box```.

In [ ]:
box = [[-1,1],[-1,1],[-1,1],[-1,1]]

In [ ]:
fpoints = np.vstack(list(map(np.array, it.product(*box))))
fhull = ConvexHull(fpoints,qhull_options='QJ')
dim = fpoints.shape[-1]

Our test function will be affine, determined by coefficients $\mathbf{a}$ and intercept $b$,
$$f(\mathbf{x}) = \mathbf{a}^T\mathbf{x} + b.$$
We encode function $f$ by an array $(\mathbf{a},b)$.

In [ ]:
ab = np.concatenate([np.ones(dim),np.zeros(1)])

and evaluate $f$ at the vertices of $\mbox{dom}(f)$.

In [ ]:
fvals = np.pad(fpoints,[(0,0),(0,1)],constant_values=1) @ ab

## -- Constraints

We will now define _test_ constraints as well. The randomly generated constraints we'll use define a convex polytope $\Omega$ inside $\mbox{dom}(f)$. 

More generally, ```doframework``` randomly generates constraints as convex polytopes within its randomly generated PWL functions' domains.

We choose a range of coordinate values within which to sample the vertices of $\Omega$.

In [ ]:
omega_range = [[-0.5,1],[-1,0.5],[-1,1],[-1,1]]

In [ ]:
omega_vertex_num = 10

We'll sample vertics for $\Omega$ within $\mbox{dom}(f)$.

In [ ]:
opoints = np.vstack(
    list(
        it.islice(
            filter(lambda point: in_domain(np.atleast_2d(point), fhull.equations, tol=tol)[0],
                box_iterator(omega_range,1)),
            omega_vertex_num)
    )
)

ovals = np.pad(opoints,[(0,0),(0,1)],constant_values=1) @ ab

## -- PWL Object

We're now ready to define a PWL object that will serve us to generate data.

A PWL object relies on a triangulation of $\mbox{dom}(f)$ that incorporates $\Omega$.

In [ ]:
fPs, fVs, oPs, oVs = triangulate(fpoints,opoints,fvals,ovals)

In [ ]:
f = PWL(fPs,fVs)

In [ ]:
ohull = ConvexHull(np.vstack(oPs),qhull_options='QJ')
constraints = np.unique(ohull.equations,axis=0)

In [ ]:
f_value_interval = [np.array(fVs).min(),np.array(fVs).max()]
f_value_range = f_value_interval[1]-f_value_interval[0]

We can use the PWL object $f$ to sample points in its domain.

In [ ]:
xs = f.sample(3)

or evaluate points

In [ ]:
f.evaluate(xs)

## -- Ground Truth

Since we have a triangulation of $f$ and $\Omega$, we also have immediate knowledge of the ground truth. We will later compare it to our naive OCL algorithm's results.

In [ ]:
argmin = np.argmin(oVs)

In [ ]:
j = argmin % oVs.shape[-1]

In [ ]:
i = int(argmin/oVs.shape[-1])

In [ ]:
opt_true = oPs[i][j]

In [ ]:
opt_true_val = f.evaluate(np.atleast_2d(opt_true))[0]

## -- Data

We'll now generate data from the test objective target $f$. The data we'll sample will be a Gaussian mix in $\mbox{dom}(f)$. 

Let's decide how many Gaussians we want in the mix.

In [ ]:
mean_num = 3

and how much noise to add to functions values in relative terms (```noise=0.05``` means $5\%$ of $f$ value range in $\mbox{dom}(f)$).

In [ ]:
noise = 0.05

We'll sample some means for the Gaussians in the mix from $\mbox{dom}(f)$.

In [ ]:
samples = f.sample(mean_num)

In [ ]:
means = [s for s in samples]

and sample some non-spherical covariance matrices.

In [ ]:
covs = [np.diag(uniform.rvs(f_value_interval[0],f_value_range,dim)**2)*np.eye(dim) for _ in range(mean_num)]

We'll also sample ```weights``` for the Gaussians in the mix.

In [ ]:
weights = sample_standard_simplex(mean_num)

We'll decide on the number of data points $N$ to sample.

In [ ]:
N = 750

and finally get some samples.

In [ ]:
D = sampler(f, N, weights, noise*(f_value_range), mean=means, cov=covs, num_cpus=4)

We'll make sure all data points are indeed in $\mbox{dom}(f)$.

In [ ]:
np.all(f.isin(D[:,:-1]))

## -- Model

Let's build a simple model class for the predict component of our OCL algorithm.

There is only _one_ requirement on the model class instance: it should have a ``predict`` method that accepts and returns ``np.array``'s.

The rest is up to us. We can add any attribute like. It will be added to the solution file generated by ``doframework``.

In [ ]:
@dataclass
class predictionModel:
    '''
    Class for the prediction model of an OCL algorithm.
    '''
        
    model = LinearRegression()
    data: InitVar[np.array] = None
    r2_score: float = field(init=False)
        
    def __post_init__(self,data):
        if data is not None:
            self.model.fit(data[:,:-1], data[:,-1])
            self.r2_score = self.model.score(data[:,:-1], data[:,-1])

    def predict(self, x: np.array) -> np.array:
        return self.model.predict(x)


In [ ]:
model = predictionModel(D)

## -- Solver

The solver class will be responsible for the optimization part of the OCL algorithm.

This particular solver class is designed to work with a simple linear regressor. It uses the [PuLP solver](https://coin-or.github.io/pulp/# "PuLP").

In [ ]:
from pulp import *

In [ ]:
@dataclass
class optimizationModel:
    '''
    Class for the solver of an OCL algorithm.
    '''

    predict: InitVar[Any]
    objective_target: np.array = field(init=False)
        
    def __post_init__(self,predict):
        self.objective_target = np.concatenate((predict.model.coef_.reshape(-1,1), 
                                                predict.model.intercept_.reshape(-1,1)))
        self.objective_target = self.objective_target.flatten()

    def optimize(self,constraints,is_minimum) -> np.array:
        
        n = self.objective_target.shape[-1]
        variables = [*range(n)]
        x = LpVariable.dicts("x", variables)

        prob = LpProblem("Optimization",LpMinimize) if is_minimum else LpProblem("Optimization",LpMaximize)
        prob += lpSum([self.objective_target[i] * x[i] for i in variables]), "Objective Target"
        prob += x[n-1] == 1, "Intercept"

        for k, eqn in enumerate(constraints):
            prob += (
                pulp.lpSum([eqn[i]*x[i] for i in variables]) <= 0,
                f"constraint_from_{k}th_facet",
            )
        prob.solve(PULP_CBC_CMD(msg=0)) # disable logs with msg=0

        return np.array([v.varValue for v in prob.variables()],dtype=np.float32)[:-1] # remove intercept coord


In [ ]:
solver = optimizationModel(model)
opt_pred = solver.optimize(constraints,is_minimum=True)
opt_pred_val = model.predict(np.atleast_2d(opt_pred))[0]    

In [ ]:
print(f'True optimum: {opt_true}\nPredicted optimum: {opt_pred}\nTrue optimal values: {opt_true_val}\nPredicted optimal value: {opt_pred_val}')

# Part II

****

## -- OCL Algorithm

Now that we tested our naive OCL algorithm, we can package it as a function that ```doframework``` can integrate into its flow.

Our function should accept ``data`` and ``constraints`` as input and produce an ``optimum`` with its predicted ``value`` as well as a ``model`` that was used in the predict phase.

In [ ]:
import doframework as dof

In [ ]:
def ocl(data: np.array, constraints: np.array, **kwargs):

    is_minimum = kwargs['is_minimum'] if 'is_minimum' in kwargs else True
                
    try:

        data_feasible = data[np.all(np.pad(data[:,:-1],((0,0),(0,1)),constant_values=1) @ constraints.T <= 0, axis=1)]
        model = predictionModel(data_feasible)
        solver = optimizationModel(model)
        optimum = solver.optimize(constraints,is_minimum)
        predicted_value = model.predict(np.atleast_2d(optimum))[0]    

    except Exception as e:
      
        print('Exception: Prediction optimization failed.')
        print(e)
        return None, None, None

    else:

        return optimum, predicted_value, model

To integrate our simple algorithm into ```doframework```, we need to **resolve** it.

In [ ]:
@dof.resolve
def ocl_resolved(data: np.array, constraints: np.array, **kwargs):
    return ocl(data, constraints, **kwargs)

## -- Configs

`doframework` relies on a configs yaml to enable its interaction with storage. Storage can be S3 buckets (AWS / IBM Cloud) or a local file system. Here, we will rely on local storage. We already uploaded `sim_configs.yaml` to our `ocl_lab` directory. Here is what it looks like:
```
local:
  buckets: 
    inputs: '../ocl_lab/data/dof-simulation/inputs'
    inputs_dest: '../ocl_lab/data/dof-simulation/inputs-dest'
    objectives: '../ocl_lab/data/dof-simulation/objectives'
    objectives_dest: '../ocl_lab/data/dof-simulation/objectives-dest'
    data: '../ocl_lab/data/dof-simulation/data'
    data_dest: '../ocl_lab/data/dof-simulation/data-dest'
    solutions: '../ocl_lab/data/dof-simulation/solutions'
```

Each `doframework` simulation product type has a _source_ bucket and a _target_ bucket underscored with `_dest`. At the end of a `doframework` run, you will find all simulation products in their `_dest` folders, except for algorithm solution files which will be under the `solutions` bucket.

We must make sure the bucket nanes we provide are **DISTINCT**. You will find all accepted configuration formats under [`doframework/configs`](https://github.com/IBM/doframework/tree/main/configs). 

In [ ]:
root = os.getcwd()
configs_file = 'sim_configs.yaml'
configs_path = os.path.join(root,configs_file)

sim_configs = get_configs(configs_path)
storage = Storage(sim_configs)
buckets = storage.buckets()

## -- Inputs

To run `doframework`, we need input json files with meta data for `doframework` to generate objectives, constraints, and datasets. Here is an example of `input_basic.json` uploaded to `ocl_lab/notebooks/DOFramework`:
```
{
    "f": {
        "vertices": {
            "num": 20,
            "range": [[0.0,10.0],[0.0,10.0],[0.0,10.0],[0.0,10.0],[0.0,10.0]]
        },
        "values": {
            "range": [-10.0,10.0]
        }
    },
    "omega": {
        "ratio": 0.6
    },
    "data": {
        "N": 1000,
        "noise": 0.01,
        "policy_num": 3,
        "scale": 0.7
    },
    "input_file_name": "input_basic.json"
}
``` 



Let's copy `input_basic.json` to the `inputs` bucket specified in `sim_configs.yaml`.




In [ ]:
input_file = 'input_basic.json'
input_path = os.path.join(buckets['inputs'],input_file)

shutil.copyfile(input_file, input_path)

This input file tells `doframework` to generate continuous piece-wise linear functions supported in `f[vertices][range]` $\subseteq \mathbb{R}^5$ with `f[vertices][num]` vertices (more vertices -> more complex functions). $f(\mathbf{x})$ values will be bounded in `f[values][range]`. 

This input file tells ```doframework``` to generate a polytope of constraints $\Omega$ that covers at least ```omega[ratio]``` of $\mbox{dom}(f)$. It tells `doframework` to generate `data[N]` points in each dataset as a Gaussian mix with `data[policy_num]` centers. The maximum length scale for each Gaussian will be at most `data[scale]` of $\mbox{dom}(f)$ diameter. Noise will be introduced to $f$ values with STD  of `data[noise]`$\cdot$`f[values][range]`.
You will find accepted input formats under [`doframework/inputs`](https://github.com/IBM/doframework/tree/main/inputs). 

## -- Run

We are finally ready to run `doframework`. `doframework` will generate a specified number of objectives, constraints (i.e., feasibility regions), and datasets, and then run them against `ocl_resolved`.

In [ ]:
objectives_num = 3
feasibility_regions_num = 1
datasets_num = 3

In [ ]:
num_inputs_at_start = storage.count(buckets['inputs'],'json')+storage.count(buckets['inputs_dest'],'json')
num_objectives_at_start = storage.count(buckets['objectives'],'json')+storage.count(buckets['objectives_dest'],'json')
num_datasets_at_start = storage.count(buckets['data'],'csv')+storage.count(buckets['data_dest'],'csv')
num_solutions_at_start = storage.count(buckets['solutions'],'json')

expected_objectives_num = objectives_num
expected_datasets_num = expected_objectives_num*datasets_num
expected_solutions_num = expected_datasets_num*feasibility_regions_num

In [ ]:
dof.run(ocl_resolved, configs_path, objectives=objectives_num, datasets=datasets_num, feasibility_regions=feasibility_regions_num, after_idle_for=20)

In [ ]:
num_inputs = storage.count(buckets['inputs'],'json')+storage.count(buckets['inputs_dest'],'json')-num_inputs_at_start
num_objectives = storage.count(buckets['objectives'],'json')+storage.count(buckets['objectives_dest'],'json')-num_objectives_at_start
num_datasets = storage.count(buckets['data'],'csv')+storage.count(buckets['data_dest'],'csv')-num_datasets_at_start
num_solutions = storage.count(buckets['solutions'],'json')-num_solutions_at_start

print(f'Generated {num_objectives} objectives out of expected {expected_objectives_num}.')
print(f'Generated {num_datasets} datasets out of expected {expected_datasets_num}.')
print(f'Generated {num_solutions} solutions out of expected {expected_solutions_num}.')